In [1]:
# -*- coding: utf-8 -*-
"""
Inverted Pendulum Lyapunov Candidate with Exact Analytical Expression

This code trains a neural network candidate Lyapunov function for an inverted pendulum
system using a multilayer network (with an arbitrary number of hidden layers). In the verification step,
the exact analytical expression for V(x) is constructed (using a symbolic MLP built with Sympy)
by reading out the trained network’s parameters. This expression is then converted into a dReal
Expression (via a recursive converter) and used to check the Lyapunov conditions.
"""

# --- Install dReal if not already installed ---
import pkgutil
if not pkgutil.find_loader("dreal"):
    # Intended for Colab or a similar environment.
    get_ipython().system('curl https://raw.githubusercontent.com/dreal/dreal4/master/setup/ubuntu/22.04/install.sh | bash')
    get_ipython().system('pip install dreal --upgrade')

# --- Import packages ---
from scipy.integrate import solve_ivp
from dreal import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import timeit
import sympy as sp
import matplotlib.pyplot as plt  # Optional for plotting

# --- Falsification and Utility Functions ---

def CheckLyapunov(x, f, V, ball_lb, ball_ub, config, epsilon):
    """
    Check the Lyapunov conditions over states x in the annulus defined by ball_lb and ball_ub.
    The conditions are:
        if x is in the ball → V >= 0   and   the Lie derivative L_V <= epsilon.
    This function uses dReal to search for a counterexample and returns one if found.
    """
    ball = Expression(0)
    lie_derivative_of_V = Expression(0)

    for i in range(len(x)):
        ball += x[i]*x[i]
        lie_derivative_of_V += f[i]*V.Differentiate(x[i])

    ball_in_bound = logical_and(ball_lb * ball_lb <= ball, ball <= ball_ub * ball_ub)

    condition = logical_and(logical_imply(ball_in_bound, V >= 0),
                            logical_imply(ball_in_bound, lie_derivative_of_V <= epsilon))

    # If dReal finds a counterexample (logical_not(condition) is SAT), then V is not valid.
    return CheckSatisfiability(logical_not(condition), config)

def AddCounterexamples(x, CE, N):
    """
    Incorporate counterexample points provided by dReal (CE) into the training set.
    For each counterexample, N random samples are generated in its box.
    """
    c = []
    nearby = []
    for i in range(CE.size()):
        c.append(CE[i].mid())
        lb = CE[i].lb()
        ub = CE[i].ub()
        nearby_ = np.random.uniform(lb, ub, N)
        nearby.append(nearby_)
    for i in range(N):
        n_pt = []
        for j in range(x.shape[1]):
            n_pt.append(nearby[j][i])
        x = torch.cat((x, torch.tensor([n_pt])), 0)
    return x

def Tune(x):
    """
    Computes a simple circle function over the training samples.
    This tuning term encourages the candidate Lyapunov function to scale with the squared norm.
    """
    y = []
    for r in range(len(x)):
        v = 0
        for j in range(x.shape[1]):
            v += x[r][j]**2
        f_val = [torch.sqrt(v)]
        y.append(f_val)
    y = torch.tensor(y)
    return y

def f_value(x, u):
    """
    Inverted pendulum dynamics.
      x: state tensor, where each row is a state [angle, angular_velocity]
      u: control input computed by the NN
    Returns: tensor of dynamics f(x,u)
    """
    y = []
    G = 9.81  # gravity
    L = 0.5   # length of the pole
    m = 0.15  # mass
    b = 0.1   # friction

    for r in range(len(x)):
        f_r = [ x[r][1],
                (m * G * L * torch.sin(x[r][0]) - b * x[r][1]) / (m * L**2) ]
        y.append(f_r)
    y = torch.tensor(y)
    # Add control influence:
    y[:, 1] = y[:, 1] + (u[:, 0] / (m * L**2))
    return y

# --- Multi-Layer Neural Network Model ---

class MultiLayerNet(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim, lqr):
        """
        input_dim: input dimension (e.g., 2 for [angle, angular_velocity])
        hidden_layers: list of ints, each specifying the number of neurons in that hidden layer
        output_dim: output dimension (usually 1 for candidate V)
        lqr: Tensor with the LQR controller solution used to initialize the control branch.
        """
        super(MultiLayerNet, self).__init__()
        self.hidden_layers = nn.ModuleList()
        current_dim = input_dim
        for h_dim in hidden_layers:
            self.hidden_layers.append(nn.Linear(current_dim, h_dim))
            current_dim = h_dim
        # Final layer for candidate V; we apply tanh as in the forward pass.
        self.output_layer = nn.Linear(current_dim, output_dim)
        # Control branch: uses the original state.
        self.control = nn.Linear(input_dim, 1, bias=False)
        self.control.weight = nn.Parameter(lqr)

    def forward(self, x):
        x_orig = x.clone()  # Save original input for control branch.
        for layer in self.hidden_layers:
            x = torch.tanh(layer(x))
        V = torch.tanh(self.output_layer(x))
        u = self.control(x_orig)
        return V, u

# --- Lie Derivative Computation using Autograd ---

def compute_lie_derivative(model, x, f):
    """
    Compute the Lie derivative L_V = ∇V(x) · f(x,u) using PyTorch autograd.
    """
    x = x.clone().detach().requires_grad_(True)
    V, _ = model(x)
    grad_V = torch.autograd.grad(outputs=V, inputs=x,
                                 grad_outputs=torch.ones_like(V),
                                 create_graph=True)[0]
    L_V = (grad_V * f).sum(dim=1)
    return L_V

# --- Construct Exact Analytical Expression from the Trained Network ---
def build_symbolic_V_from_model(model, input_dim):
    """
    Constructs a symbolic expression V(x) that exactly represents the current
    multilayer network (with tanh activation at every layer). Uses Sympy.
    Returns a tuple (x_syms, V_sym) where:
      - x_syms: a tuple of Sympy symbols for the inputs,
      - V_sym: a Sympy expression for V.
    """
    # Define input symbols x1, x2, ..., x{input_dim}
    x_syms = sp.symbols(f'x1:{input_dim+1}')
    weights = {}
    biases = {}
    layer_index = 1
    # Process the hidden layers.
    for layer in model.hidden_layers:
        W = layer.weight.detach().numpy()  # Shape: (out_features, in_features)
        b = layer.bias.detach().numpy()      # Shape: (out_features,)
        out_features, in_features = W.shape
        for k in range(out_features):
            for j in range(in_features):
                weights[(layer_index, j+1, k+1)] = sp.nsimplify(W[k, j])
            biases[(layer_index, k+1)] = sp.nsimplify(b[k])
        layer_index += 1
    # Process the output layer.
    W = model.output_layer.weight.detach().numpy()
    b = model.output_layer.bias.detach().numpy()
    out_features, in_features = W.shape
    for k in range(out_features):
        for j in range(in_features):
            weights[(layer_index, j+1, k+1)] = sp.nsimplify(W[k, j])
        biases[(layer_index, k+1)] = sp.nsimplify(b[k])
    num_layers = len(model.hidden_layers) + 1

    def mlp_forward_all_tanh(x_list, weights, biases, num_layers):
        layer_input = list(x_list)
        for i in range(1, num_layers + 1):
            # Get all neuron indices in layer i.
            neurons = [k for (layer, k) in biases.keys() if layer == i]
            neurons.sort()
            layer_output = []
            for k in neurons:
                neuron_inp = sum(weights[(i, j, k)] * layer_input[j-1] for j in range(1, len(layer_input) + 1)) + biases[(i, k)]
                neuron_out = sp.tanh(neuron_inp)
                layer_output.append(neuron_out)
            layer_input = layer_output
        # Assume single output neuron.
        return layer_input[0]

    V_sym = mlp_forward_all_tanh(x_syms, weights, biases, num_layers)
    return x_syms, V_sym

# --- Helper: Convert a Sympy Expression to a dReal Expression ---
def sympy_to_dreal(expr, var_mapping):
    """
    Recursively converts a Sympy expression into a dReal Expression.
    var_mapping is a dictionary mapping string names of Sympy symbols to dReal Variables.
    Supports basic arithmetic and functions: +, -, *, ** (with numeric exponents),
    tanh, sin, cos, log, exp, and sqrt.
    """
    # Base case: number
    if expr.is_Number:
        return Expression(float(expr))
    # Variable
    if expr.is_Symbol:
        name = str(expr)
        if name in var_mapping:
            return var_mapping[name]
        else:
            raise ValueError(f"Symbol {name} not found in variable mapping.")
    # Addition
    if expr.is_Add:
        result = sympy_to_dreal(expr.args[0], var_mapping)
        for arg in expr.args[1:]:
            result = result + sympy_to_dreal(arg, var_mapping)
        return result
    # Multiplication
    if expr.is_Mul:
        result = sympy_to_dreal(expr.args[0], var_mapping)
        for arg in expr.args[1:]:
            result = result * sympy_to_dreal(arg, var_mapping)
        return result
    # Power (only with numeric exponent supported)
    if expr.is_Pow:
        base = sympy_to_dreal(expr.args[0], var_mapping)
        exponent = expr.args[1]
        if exponent.is_Number:
            return base ** float(exponent)
        else:
            raise NotImplementedError("Non-numeric exponent in expression: " + str(expr))
    # Common functions:
    if expr.func == sp.tanh:
        arg = sympy_to_dreal(expr.args[0], var_mapping)
        return tanh(arg)
    if expr.func == sp.sin:
        arg = sympy_to_dreal(expr.args[0], var_mapping)
        return sin(arg)
    if expr.func == sp.cos:
        arg = sympy_to_dreal(expr.args[0], var_mapping)
        return cos(arg)
    if expr.func == sp.log:
        arg = sympy_to_dreal(expr.args[0], var_mapping)
        return log(arg)
    if expr.func == sp.exp:
        arg = sympy_to_dreal(expr.args[0], var_mapping)
        return exp(arg)
    if expr.func == sp.sqrt:
        arg = sympy_to_dreal(expr.args[0], var_mapping)
        return arg ** 0.5
    raise NotImplementedError("Conversion not implemented for: " + str(expr))

# --- Options for Learning and Verification ---

# Learning options:
N = 500             # Number of training samples
D_in = 2            # Input dimension
# hidden_layers = [2, 3]  # You may extend this list to add more hidden layers
hidden_layers = [6]  # You may extend this list to add more hidden layers
D_out = 1           # Candidate V is scalar
torch.manual_seed(10)
x = torch.Tensor(N, D_in).uniform_(-6, 6).float()
x_0 = torch.zeros([1, D_in]).float()

# Verification options (using dReal Variables):
# Create dReal variables "x1", "x2", ... (matching our symbolic input names)
x1 = Variable("x1")
x2 = Variable("x2")
vars_ = [x1, x2]

G_const = 9.81
l_const = 0.5
m_const = 0.15
b_const = 0.1
config = Config()
config.use_polytope_in_forall = True
config.use_local_optimization = True
config.precision = 1e-2
epsilon = 0

# Domain for verification (annulus):
ball_lb = 0.5
ball_ub = 6

# --- Learning and Falsification Loop ---
out_iters = 0
valid = False

while out_iters < 2 and not valid:
    start = timeit.default_timer()
    lqr = torch.tensor([[-23.58639732, -5.31421063]])  # LQR solution for control branch
    model = MultiLayerNet(D_in, hidden_layers, D_out, lqr)
    L_history = []
    i = 0
    verified_time = 0
    max_iters = 2000
    learning_rate = 0.01
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    while i < max_iters and not valid:
        # Forward pass: compute candidate V and control input u for training samples.
        V_candidate, u = model(x)
        X0, _ = model(x_0)
        f_dyn = f_value(x, u)
        L_V = compute_lie_derivative(model, x, f_dyn)
        Circle_Tuning = Tune(x)

        Lyapunov_risk = (F.relu(-V_candidate) + 1.5 * F.relu(L_V + 0.5)).mean() \
                        + 2.2 * ((Circle_Tuning - 6 * V_candidate).pow(2)).mean() \
                        + (X0).pow(2)
        print(i, "Lyapunov Risk =", Lyapunov_risk.item())
        L_history.append(Lyapunov_risk.item())

        optimizer.zero_grad()
        Lyapunov_risk.backward()
        optimizer.step()

        # Every 10 iterations, perform symbolic verification using dReal.
        if i % 10 == 0:
            # Build symbolic control input using the LQR branch.
            q_val = model.control.weight.detach().numpy()
            u_NN = q_val[0][0] * x1 + q_val[0][1] * x2

            # Define symbolic dynamics for verification using dReal functions.
            f_symbolic = [ x2,
                (m_const * G_const * l_const * sin(x1) + u_NN - b_const * x2) / (m_const * l_const**2) ]

            # Build the exact analytical candidate V symbolically from the network.
            x_syms, V_sym = build_symbolic_V_from_model(model, D_in)
            # Create a mapping from Sympy variable names to dReal Variables.
            var_mapping = { f'x{i+1}': vars_[i] for i in range(len(vars_)) }
            V_exact = sympy_to_dreal(V_sym, var_mapping)

            print('=========== Verifying ==========')
            start_verif = timeit.default_timer()
            result = CheckLyapunov(vars_, f_symbolic, V_exact, ball_lb, ball_ub, config, epsilon)
            stop_verif = timeit.default_timer()

            if result:
                print("Not a Lyapunov function. Found counterexample:")
                print(result)
                # Add counterexample points into the training set.
                x = AddCounterexamples(x, result, 10)
                x = x.float()
            else:
                valid = True
                print("Satisfies conditions!!")
                print("Exact V(x) =", V_exact)
            verified_time += (stop_verif - start_verif)
            print('================================')
        i += 1

    stop = timeit.default_timer()
    # Optionally, save network parameters.
    np.savetxt("w1.txt", model.hidden_layers[0].weight.detach().numpy(), fmt="%s")
    np.savetxt("w2.txt", model.output_layer.weight.detach().numpy(), fmt="%s")
    np.savetxt("b1.txt", model.hidden_layers[0].bias.detach().numpy(), fmt="%s")
    np.savetxt("b2.txt", model.output_layer.bias.detach().numpy(), fmt="%s")
    np.savetxt("q.txt", model.control.weight.detach().numpy(), fmt="%s")

    print('\nTotal training time:', stop - start)
    print("Total verification time:", verified_time)
    out_iters += 1

# --- Final Verification with a Smaller Epsilon ---
epsilon = -0.00001
start_final = timeit.default_timer()
result = CheckLyapunov(vars_, f_symbolic, V_exact, ball_lb, ball_ub, config, epsilon)
stop_final = timeit.default_timer()

if result:
    print("Not a Lyapunov function. Found counterexample:")
    print(result)
else:
    print("Satisfies conditions with epsilon =", epsilon)
    print("Exact V(x) =", V_exact)


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1754  100  1754    0     0   4131      0 --:--:-- --:--:-- --:--:--  4127
+ [[ 0 -ne 0 ]]
+ apt-get install -y --no-install-recommends software-properties-common
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
software-properties-common is already the newest version (0.99.22.9).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
+ add-apt-repository ppa:dreal/dreal --no-update -y
Repository: 'deb https://ppa.launchpadcontent.net/dreal/dreal/ubuntu/ jammy main'
Description:
Delta-complete SMT Solver and its tools. Please visit https://dreal.github.io for more information.
More info: https://launchpad.net/~dreal/+archive/ubuntu/dreal
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/dreal-ubuntu-dreal-jammy.list
Adding disabled deb-src entry to /etc/apt/